In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# EDA
## Weather Data Import

In [ ]:
weather = pd.read_csv('data/SummaryOfWeather.csv', low_memory=False)

In [ ]:
## Get the number of columns and rows
weather.shape

### Data Transformations Pre-Analysis

#### Dealing w/Null Data

In [ ]:
### How many null columns
weather.isnull().sum()

#### Temp Conversions

In [ ]:
weather['MaxTemp'] = weather['MaxTemp'].apply(lambda x: int((x * 9/5) + 32))
weather['MinTemp'] = weather['MinTemp'].apply(lambda x: int((x * 9/5) + 32))
weather['MeanTemp'] = weather['MeanTemp'].apply(lambda x: int((x * 9/5) + 32))

## Data Analysis Summary

- The weather dataset has 31 parameters and 119,040 observations
- We will drop the following columns: 

**Null Summary**

- **STA** - Weather Station
- **Date** - Self-explanatory
- **Precip** - Precipitation in mm
- **WindGustSpd** - Peak wind gust speed in km/h 
    - **118,508** missing observations
    - Keep windgust speeds for now but leave out of model
    - Set null values to 0 for now. 
- **MaxTemp** - Maximum temperature in degrees Celsius
    - Convert to Farenheit **(0°C × 9/5) + 32 = 32°F)**
- **MinTemp** - Minimum temperature in degrees Celsius
    - Convert to Farenheit
- **MeanTemp** - Mean temperature in degrees Celsius
    - Convert to Farenheit
- **Snowfall** - Snowfall and Ice Pellets in mm 
    - **1163** missing observations
    - Keep snowfall
- **PoorWeather** -A repeat of the TSHDSBRSGF column 
    - **84803** missing observations
    - Keep PoorWeather and add ass a factor
    - 1 = Yes, 0 = Unknown
    - Determine if we can find this data or derive it based on temperature and precipitation
- **YR** - Year of Observation
- **MO** - Month of Observation
- **DA** - Day of Observation
- **PRCP** - Precipitation in Inches and Hundredths 
    - **1932** missing observations
    - Keep and use to determine weather conditions
-  **DR** - Peak wind gust direction in tens of degrees 
    - **118507** missing observations
    - Set null values to 0 for now
- **SPD** - Peak wind gust speed in knots 
    - **118508** missing observations
    - Set null values to 0 for now
- **MAX** - Maximum temperature in degrees Fahrenheit 
    - **474** missing observations
    - See what days are missing and see if they correlate to MIN, MAX. Also determine if important missions happended during this time. 
- **MIN** - Minimum temperature in degrees Fahrenheit 
    - **468** missing observations
    - Same as max
- **MEA** - Mean temperature in degrees Fahrenheit 
    - **498** missing observations
    - Same as Max
- **SNF** - Snowfall in inches and tenths 
    - **1163** missing observations
    - Set unknown to 0 for now
- **SND** - Snow depth (includes ice pellets) recorded at 1200 GMT except 0000 GMT in Far East Asian Area in inches and tenths 
    - **113477** missing observations
    - Set nulls to zero
- **FTFrozen** - Ground Top (depth in inches) 
    - **119040** missing observations
    - Delete for now unless we can find the data
- **FBFrozen** - Ground Base (depth in inches) 
    - **119040** missing observations
    - Delete for now unless we can find the data
- **FTIFrozen** - Ground Thickness (thickness in inches) 
    - **119040** missing observations
    - Delete for now unless we can find the data
- **ITHIce** - Thickness on Water (inches and tenths) 
    - **119040** missing observations
    - Delete for now unless we can find the data
- **PGTPeak** wind gust time (hours and tenths) 
    - **118515** missing observations
    - Set null values to 0 for now
- **TSHDSBRSGFDay** with: Thunder; Sleet; Hail; Dust or Sand; Smoke or Haze; Blowing Snow; Rain; Snow; Glaze; Fog; 0 = No, 1 = Yes 
    - **84803** missing observations
    - These are all events that signify a weather event that may not be conducive 
    - All Null values will be placed with a 0 for now.  
- **SD3Snow** depth at 0030 GMT includes ice pellets in inches and tenths
    - **119040** missing observations
    - Delete for now unless we can find the data
- **RHX24-hour** maximum relative humidity, as a whole percent 
    - **119040** missing observations
    - Delete for now unless we can find the data
- **RHN24-hour** minimum relative humidity, as a whole percent 
    - **119040** missing observations
    - Delete for now unless we can find the data
- **RVGRiver** guage in feet and tenths 
    - **119040** missing observations
    - Delete for now unless we can find the data
- **WTEWater** equivalent of snow and ice on ground in inches and hundredths 
    - **(119040)** missing observations
    - Delete for now unless we can find the data
    

In [ ]:
weather.drop(['WTE','RVG','RHN','RHX','SD3','ITH','FTI','FB','FT'], axis=1, inplace=True)

### Detailed Summary

In [ ]:
columns = ['WindGustSpd','MaxTemp','MinTemp','MeanTemp','YR','MO','DA','DR','SPD','MAX','MIN','MEA','SND','PGT']
weather[columns].describe().round(0)

### STA - Weather Station

In [ ]:
## There are 159 unique weather stations.  We should focus on analysis by station.  
num_bins = 159
plt.hist(weather['STA'], num_bins, density=1, facecolor='blue', alpha=0.5)
plt.show()

#weather.groupby('STA')['STA'].count()

In [ ]:
## Which weather stations have the most data
weather.groupby('STA').size().sort_values(ascending=False)

In [ ]:
sta_gb = weather['MaxTemp'].groupby(weather['STA'])
sta_gb.describe()
#weather.groupby(weather['STA']).describe()

In [ ]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}

bins = [0, 32, 45,70, 85, 120]
group_names = ['Freezing','Cold','Mild','Warm','Hot']


### Temperature Categories
#### MeanTemp Analysis

In [ ]:
weather['categories'] = pd.cut(weather['MeanTemp'], bins, labels=group_names)
weather['MeanTemp'].groupby(weather['categories']).apply(get_stats).unstack()

#### MinTemp Analysis

In [ ]:
weather['categories'] = pd.cut(weather['MinTemp'], bins, labels=group_names)
weather['MinTemp'].groupby(weather['categories']).apply(get_stats).unstack()

#### MaxTemp Analysis

In [ ]:
weather['categories'] = pd.cut(weather['MaxTemp'], bins, labels=group_names)
weather['MaxTemp'].groupby(weather['categories']).apply(get_stats).unstack()

- Let's take a hard look at the locations in respect to the density of weather stations.  We'll definitely need to determine the proximity of the stations to the missions.  

## Weather Stations Data

In [ ]:
stations = pd.read_csv('data/WeatherStationLocations.csv')
country = pd.read_csv('data/Country.csv')
print(stations.head(),"\n")
print(country.head())
df = pd.merge(stations, country, on='COUNTRYID', how='outer')
print(df.head())

In [ ]:
df[df.WBAN==10701]

### [GeoDataFrame](https://geopandas.readthedocs.io/en/latest/gallery/create_geopandas_from_pandas.html)

In [ ]:
import geopandas
from shapely.geometry import Point
#Creating a GeoDataFrame from a DataFrame with coordinates


## A GeoDataFrame needs a shapely object, so we create a new column Coordinates 
## as a tuple of Longitude and Latitude :
stations['Coordinates'] = list(zip(stations.Longitude, stations.Latitude))

## Then, we transform tuples to Point :
stations['Coordinates'] = stations['Coordinates'].apply(Point)

# Now, we can create the GeoDataFrame by setting geometry with the coordinates created previously.
gdf = geopandas.GeoDataFrame(stations, geometry='Coordinates')
print(gdf.head())


In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
base = world.plot(color='white', edgecolor='black', figsize=(20,20))
gdf.plot(ax=base, color='red')
plt.show()

In [ ]:
world.plot()

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

In [ ]:
data.head()

## Load Data

In [110]:
### Extract data file from zip files

def extract_txt(path, storage_path, f):
    # Now change the directory
    os.chdir( path )
    import zipfile
    zipfile.ZipFile
    #print(f)
    my_zip = zipfile.ZipFile(f) # Specify your zip file's name here
    storage_path = storage_path
    for file in my_zip.namelist():
        #if my_zip.getinfo(file).filename.endswith('.txt'):
        if my_zip.getinfo(file).filename.startswith('produkt'):
            print("unzipping:", file)
            my_zip.extract(file, storage_path)
        

In [111]:
from google.cloud import bigquery as bq
#ml-jth-d220ef33563e.json


def load_bq(dataset, table, data):
    client = bq.Client.from_service_account_json('/Users/jharding/repos/SMU-DS-6372/ml-jth-d220ef33563e.json')
    dataset_id = dataset
    
    dataset_ref = client.dataset(dataset_id)
    table_ref = table
    
    job_config=bigquery.LoadJobConfig()
    job_config.schema = [
        bq.SchemaField('StationID', 'INTEGER'),
        bq.SchemaField('Date', 'DATE'),
        bq.SchemaField('QN3',  'INTEGER'),
        bq.SchemaField('MaxWindGust', 'FLOAT64'),
        bq.SchemaField('MeanWindSpeed', 'FLOAT64'),
        bq.SchemaField('QN4', 'INT64'),
        bq.SchemaField('PrecipHeight', 'FLOAT64'),
        bq.SchemaField('PrecipForm', 'INT64'),
        bq.SchemaField('SunDuration', 'FLOAT64'),
        bq.SchemaField('SnowDepth', 'INT64'),
        bq.SchemaField('MeanCloudCover', 'FLOAT64'),
        bq.SchemaField('MeanCloudVapor', 'FLOAT64'),
        bq.SchemaField('MeanPressure', 'FLOAT64'),
        bq.SchemaField('MeanTemp', 'FLOAT64'),
        bq.SchemaField('MeanRelHumid', 'FLOAT64'),
        bq.SchemaField('MaxTemp', 'FLOAT64'),
        bq.SchemaField('MinTemp', 'FLOAT64'),
        bq.SchemaField('MinAirTemp', 'FLOAT64')  
    ]
    
    uri = data
                       
    load_job = client.load_table_from_dataframe(
        uri,
        dataset_ref.table(table_ref),
        job_config=job_config)
    
    print('Starting job{}',format(load_job.job_id))
    
    load_job.result()
    print('Job Finished\n')
    

In [112]:
def extract_data(file):
    test = pd.read_csv(file,sep=';')
    test.columns = ['StationID','DateTime','QN3','MaxWindGust',
                'MeanWindSpeed','QN4','PrecipHeight','PrecipForm',
                'SunDuration', 'SnowDepth', 'MeanCloudCover',
                'MeanCloudVapor', 'MeanPressure','MeanTemp',
                'MeanRelHumid','MaxTemp','MinTemp','MinAirTemp','eor']

    ## Change to Date Format
    dateFormat = '%Y%m%d'
    test['DateTime'] = pd.to_datetime(test['DateTime'], format=dateFormat)
    test['Date']=test['DateTime'].dt.date
    
    ## Drop unneeded columns
    test.drop(['eor','DateTime'], axis=1, inplace=True)
    
    ## Force dataframe to Float64
    test['MaxWindGust'] = test['MaxWindGust'].astype(np.float64)
    test['MeanPressure'] = test['MeanPressure'].astype(np.float64)
    test['MeanWindSpeed'] = test['MeanWindSpeed'].astype(np.float64)
    test['MinAirTemp'] = test['MinAirTemp'].astype(np.float64)
    test['PrecipHeight'] = test['PrecipHeight'].astype(np.float64)
    test['SunDuration'] = test['SunDuration'].astype(np.float64)
    test['MeanCloudCover'] = test['MeanCloudCover'].astype(np.float64)
    test['MeanCloudVapor'] = test['MeanCloudVapor'].astype(np.float64)
    test['MeanTemp'] = test['MeanTemp'].astype(np.float64)
    test['MeanRelHumid'] = test['MeanRelHumid'].astype(np.float64)
    test['MaxTemp'] = test['MaxTemp'].astype(np.float64)
    test['MinTemp'] = test['MinTemp'].astype(np.float64)
    
    print("Preparing to load:",file)
    load_bq('germany', 'weather', test)


In [113]:
### Get all of the zip files and put them in a list

from __future__ import print_function
import os

def get_file(path, ext):
    z=[]   
    files = os.listdir(path)
    for name in files:
        if name.endswith(ext):
            z.append(name)
    return z
            



In [114]:
## Set Path
storage_path = '/Users/jharding/repos/SMU-DS-6372/project/DS6372/data/dest/'
path = '/Users/jharding/repos/SMU-DS-6372/project/DS6372/data/zip/'

## Get list of zip files.
z = get_file(path, '.zip')

## For each zip file, extract text file.
path = '/Users/jharding/repos/SMU-DS-6372/project/DS6372/data/zip/'
for file in z:
    extract_txt(path, storage_path, file)
    



unzipping: produkt_klima_tag_19850801_19900531_00221.txt
unzipping: produkt_klima_tag_19360101_20031231_05250.txt
unzipping: produkt_klima_tag_19470101_20171231_05490.txt
unzipping: produkt_klima_tag_20060301_20171231_07373.txt
unzipping: produkt_klima_tag_19580101_20171231_04928.txt
unzipping: produkt_klima_tag_19370101_20040630_00717.txt
unzipping: produkt_klima_tag_19490101_19631231_01621.txt
unzipping: produkt_klima_tag_20000401_20171231_06186.txt
unzipping: produkt_klima_tag_19910101_19970331_01410.txt
unzipping: produkt_klima_tag_19470101_20010531_01182.txt
unzipping: produkt_klima_tag_19911001_20171231_03083.txt
unzipping: produkt_klima_tag_19760601_20171231_00760.txt
unzipping: produkt_klima_tag_19490101_20171231_01468.txt
unzipping: produkt_klima_tag_19850701_20061231_01378.txt
unzipping: produkt_klima_tag_20081201_20171231_13965.txt
unzipping: produkt_klima_tag_20040601_20171231_00617.txt
unzipping: produkt_klima_tag_19821101_20171231_01832.txt
unzipping: produkt_klima_tag_19

unzipping: produkt_klima_tag_19060401_20171231_03015.txt
unzipping: produkt_klima_tag_19450101_20061231_00480.txt
unzipping: produkt_klima_tag_19711101_20061231_05185.txt
unzipping: produkt_klima_tag_20080928_20081229_13904.txt
unzipping: produkt_klima_tag_19980101_20070828_04719.txt
unzipping: produkt_klima_tag_19510101_19601231_04396.txt
unzipping: produkt_klima_tag_19871201_20171231_05158.txt
unzipping: produkt_klima_tag_19370101_20171231_00330.txt
unzipping: produkt_klima_tag_19950601_19960930_02000.txt
unzipping: produkt_klima_tag_19670101_20171231_01803.txt
unzipping: produkt_klima_tag_19670101_19801231_02475.txt
unzipping: produkt_klima_tag_20160501_20171231_15555.txt
unzipping: produkt_klima_tag_20010506_20021117_01515.txt
unzipping: produkt_klima_tag_19470101_20171231_01544.txt
unzipping: produkt_klima_tag_19510301_20171231_05149.txt
unzipping: produkt_klima_tag_18910101_19740630_02374.txt
unzipping: produkt_klima_tag_19510101_20000630_02327.txt
unzipping: produkt_klima_tag_19

unzipping: produkt_klima_tag_19470101_19561231_03076.txt
unzipping: produkt_klima_tag_18980301_20171231_01757.txt
unzipping: produkt_klima_tag_19610101_20051231_14311.txt
unzipping: produkt_klima_tag_19931001_20171231_02704.txt
unzipping: produkt_klima_tag_19950401_20041130_03370.txt
unzipping: produkt_klima_tag_19480101_19571231_01428.txt
unzipping: produkt_klima_tag_19510101_19910731_05890.txt
unzipping: produkt_klima_tag_19470101_19921231_00524.txt
unzipping: produkt_klima_tag_20040601_20171231_06259.txt
unzipping: produkt_klima_tag_19910101_20171231_01411.txt
unzipping: produkt_klima_tag_19530701_20171231_05825.txt
unzipping: produkt_klima_tag_19630101_19751231_02053.txt
unzipping: produkt_klima_tag_19760101_20021031_03576.txt
unzipping: produkt_klima_tag_19610101_20171231_02953.txt
unzipping: produkt_klima_tag_19000101_19770630_04933.txt
unzipping: produkt_klima_tag_19490101_20061231_02268.txt
unzipping: produkt_klima_tag_18900101_20171231_04625.txt
unzipping: produkt_klima_tag_19

unzipping: produkt_klima_tag_19740501_19930331_03300.txt
unzipping: produkt_klima_tag_20050908_20171231_07351.txt
unzipping: produkt_klima_tag_19510101_19801231_04765.txt
unzipping: produkt_klima_tag_20060601_20171231_07394.txt
unzipping: produkt_klima_tag_19010101_20171231_02486.txt
unzipping: produkt_klima_tag_20040901_20171231_06266.txt
unzipping: produkt_klima_tag_19560601_20171231_03234.txt
unzipping: produkt_klima_tag_20050715_20171231_07341.txt
unzipping: produkt_klima_tag_19980101_20171231_03897.txt
unzipping: produkt_klima_tag_19480101_19990831_05644.txt
unzipping: produkt_klima_tag_19530701_19661231_02289.txt
unzipping: produkt_klima_tag_19660101_19931031_00642.txt
unzipping: produkt_klima_tag_19520101_19801231_03160.txt
unzipping: produkt_klima_tag_19470801_20020131_02949.txt
unzipping: produkt_klima_tag_19610701_19680430_01586.txt
unzipping: produkt_klima_tag_19610101_20171231_00400.txt
unzipping: produkt_klima_tag_19470401_19730228_01074.txt
unzipping: produkt_klima_tag_20

unzipping: produkt_klima_tag_19550101_20171231_01161.txt
unzipping: produkt_klima_tag_19360801_20001130_01076.txt
unzipping: produkt_klima_tag_19410101_20000630_04664.txt
unzipping: produkt_klima_tag_19370203_19740731_00919.txt
unzipping: produkt_klima_tag_19931101_19941231_00643.txt
unzipping: produkt_klima_tag_19610101_19810331_05675.txt
unzipping: produkt_klima_tag_19850301_20171231_03086.txt
unzipping: produkt_klima_tag_19690701_20171231_01078.txt
unzipping: produkt_klima_tag_19450101_20171231_02968.txt
unzipping: produkt_klima_tag_19400101_20051231_05582.txt
unzipping: produkt_klima_tag_19520501_20171231_02115.txt
unzipping: produkt_klima_tag_19470101_19511231_04436.txt
unzipping: produkt_klima_tag_19530101_19840630_05429.txt
unzipping: produkt_klima_tag_19870101_20041231_04433.txt
unzipping: produkt_klima_tag_19890801_20031231_00931.txt
unzipping: produkt_klima_tag_19540601_20171231_03379.txt
unzipping: produkt_klima_tag_19690501_19911130_00172.txt
unzipping: produkt_klima_tag_19

unzipping: produkt_klima_tag_19831101_20041231_03139.txt
unzipping: produkt_klima_tag_19360101_19750531_02069.txt
unzipping: produkt_klima_tag_19530101_19961231_04053.txt
unzipping: produkt_klima_tag_19620401_20061231_02950.txt
unzipping: produkt_klima_tag_20041101_20171231_06312.txt
unzipping: produkt_klima_tag_19510101_19891130_01172.txt
unzipping: produkt_klima_tag_19460101_20171231_00891.txt
unzipping: produkt_klima_tag_19490101_20171231_02750.txt
unzipping: produkt_klima_tag_19490101_19840430_01638.txt
unzipping: produkt_klima_tag_20041001_20171231_06272.txt
unzipping: produkt_klima_tag_19670801_20061231_00502.txt
unzipping: produkt_klima_tag_19510101_19591231_02048.txt
unzipping: produkt_klima_tag_20110901_20171231_00161.txt
unzipping: produkt_klima_tag_19750101_20171231_03667.txt
unzipping: produkt_klima_tag_19490101_19601231_01849.txt
unzipping: produkt_klima_tag_19480101_20171231_04709.txt
unzipping: produkt_klima_tag_20071101_20171231_07368.txt
unzipping: produkt_klima_tag_19

unzipping: produkt_klima_tag_20160901_20171231_01886.txt
unzipping: produkt_klima_tag_19861201_19900430_00273.txt
unzipping: produkt_klima_tag_19530301_20080229_02206.txt
unzipping: produkt_klima_tag_19700101_20020531_02122.txt
unzipping: produkt_klima_tag_19910101_20171231_03289.txt
unzipping: produkt_klima_tag_19570901_20171231_02667.txt
unzipping: produkt_klima_tag_19810401_20171231_05676.txt
unzipping: produkt_klima_tag_19580901_20061231_00243.txt
unzipping: produkt_klima_tag_20071101_20171231_07369.txt
unzipping: produkt_klima_tag_20070901_20171231_07427.txt
unzipping: produkt_klima_tag_18900101_19730630_03088.txt
unzipping: produkt_klima_tag_19460101_19800630_02775.txt
unzipping: produkt_klima_tag_19470101_19611231_02915.txt
unzipping: produkt_klima_tag_19930701_20171231_03042.txt
unzipping: produkt_klima_tag_19740901_19810331_02609.txt
unzipping: produkt_klima_tag_19400101_19920516_03390.txt
unzipping: produkt_klima_tag_19891001_20120930_03791.txt
unzipping: produkt_klima_tag_19

In [ ]:
#### Load txt files
path = '/Users/jharding/repos/SMU-DS-6372/project/DS6372/data/dest'
os.chdir( path )
t = get_file(path, '.txt')

for file in t:
    extract_data(file)


Preparing to load: produkt_klima_tag_19400101_19951231_03096.txt
Starting job{} 5d14d6b7-faf8-4b38-9665-a799058c8aa5
Job Finished

Preparing to load: produkt_klima_tag_20071101_20171231_07368.txt
Starting job{} 194887e1-c336-493f-811d-6413dfcf83ff
Job Finished

Preparing to load: produkt_klima_tag_19860601_20171231_02794.txt
Starting job{} 39e2b5f1-6da7-492c-9b03-06fff5ffef2d
Job Finished

Preparing to load: produkt_klima_tag_19650101_19911216_00888.txt
Starting job{} b38d1990-b3f7-47f9-b00a-c00d41d3406b
Job Finished

Preparing to load: produkt_klima_tag_19550101_20031231_00657.txt
Starting job{} 02f7b7be-f801-442f-a4fb-09da5cb2396f
Job Finished

Preparing to load: produkt_klima_tag_19470101_19860430_00772.txt
Starting job{} 444a9965-a031-41ac-916e-f4b1ca1060eb
Job Finished

Preparing to load: produkt_klima_tag_19460701_19770630_02533.txt
Starting job{} c8ca3f38-da49-4b4a-a8ad-4feb756f3825
Job Finished

Preparing to load: produkt_klima_tag_19730601_19860930_00070.txt
Starting job{} 0e4

Job Finished

Preparing to load: produkt_klima_tag_19950306_20171231_03660.txt
Starting job{} 1cdef76c-f832-4ea2-881c-b85a70755055
Job Finished

Preparing to load: produkt_klima_tag_19940501_20070531_01084.txt
Starting job{} ef0c0cc1-b71e-4fc4-ba4f-b6e61df66325
Job Finished

Preparing to load: produkt_klima_tag_19470101_20171231_00298.txt
Starting job{} 08e181ec-ee4f-4908-8181-8c6fd3896fbb
Job Finished

Preparing to load: produkt_klima_tag_19901101_20171231_04857.txt
Starting job{} eb963e98-305d-4e42-bbba-cd115c4373ad
Job Finished

Preparing to load: produkt_klima_tag_19450101_20061130_04125.txt
Starting job{} efaadba0-bd95-455b-8ac2-9314722bfcb0
Job Finished

Preparing to load: produkt_klima_tag_20110201_20171231_04154.txt
Starting job{} 57a54bad-8db5-43c5-a5bd-091df907e311
Job Finished

Preparing to load: produkt_klima_tag_20071101_20171231_07369.txt
Starting job{} 91d600f2-762d-4349-9a62-3b5d09383505
Job Finished

Preparing to load: produkt_klima_tag_19930501_20070630_03045.txt
Star

Starting job{} 9dbbdbff-9837-4e1d-afb5-641a93384367
Job Finished

Preparing to load: produkt_klima_tag_19980101_20171231_02023.txt
Starting job{} dcc1325f-9730-4d66-9c38-76837a84cdc6
Job Finished

Preparing to load: produkt_klima_tag_19931101_19941231_00643.txt
Starting job{} 2d7b3038-7825-4b36-905d-dcdd0d79377f
Job Finished

Preparing to load: produkt_klima_tag_19360101_20021231_04741.txt
Starting job{} 18658f48-0a69-4d99-8186-c7c6dd362085
Job Finished

Preparing to load: produkt_klima_tag_19510401_19691031_04391.txt
Starting job{} 82af6f3d-bf84-40d7-b1b1-a26d2ccd5a40
Job Finished

Preparing to load: produkt_klima_tag_19920601_20031231_05453.txt
Starting job{} 6e69ffdf-f02c-4899-8297-170b354fb57e
Job Finished

Preparing to load: produkt_klima_tag_19510101_20171231_02323.txt
Starting job{} b5e3aa3d-037a-427a-8326-ad8f16a8da7d
Job Finished

Preparing to load: produkt_klima_tag_19480101_19920930_04240.txt
Starting job{} 66caa5ae-444b-41da-8c76-ba1b42cc0277
Job Finished

Preparing to load

Starting job{} a943e610-7c57-44df-ad08-59ce777d95ce
Job Finished

Preparing to load: produkt_klima_tag_19510101_20021031_04581.txt
Starting job{} 485d179d-8528-4a0d-ad68-2be0321e7917
Job Finished

Preparing to load: produkt_klima_tag_19530901_20171231_04175.txt
Starting job{} be3ca043-3e50-4456-877d-bdfabec6afab
Job Finished

Preparing to load: produkt_klima_tag_19840301_19871231_00174.txt
Starting job{} 51c761b5-85bc-4930-8998-ca03ae960f9f
Job Finished

Preparing to load: produkt_klima_tag_19940801_20020331_05609.txt
Starting job{} 036bcac1-0edf-42ac-bceb-87adbf584257
Job Finished

Preparing to load: produkt_klima_tag_19360101_20070831_03537.txt
Starting job{} c38ba6f4-9d78-454c-b66a-83189dd3bae9
Job Finished

Preparing to load: produkt_klima_tag_20040601_20171231_06273.txt
Starting job{} 4fcc9f55-fd3c-40b2-b481-ffb8841ef576
Job Finished

Preparing to load: produkt_klima_tag_19510101_19840731_04927.txt
Starting job{} 7e35d459-6681-46df-8a9b-4622afd0df7d
Job Finished

Preparing to load

Starting job{} d2a1f4dd-91b1-49d6-8bdf-09a7f11893b5
Job Finished

Preparing to load: produkt_klima_tag_19981101_20171231_05871.txt
Starting job{} bb1dd3c1-470d-4a38-9885-bfac3a17bbaf
Job Finished

Preparing to load: produkt_klima_tag_19770401_19810731_04576.txt
Starting job{} ad71f5ce-28c4-43e1-85b8-2cc1af0c7ea1
Job Finished

Preparing to load: produkt_klima_tag_20060801_20171231_07350.txt
Starting job{} c56a474b-906f-4c8d-8660-9537bf107a20
Job Finished

Preparing to load: produkt_klima_tag_20000401_20171231_06186.txt
Starting job{} d6e2640d-a6c2-4ff5-9318-c4bb1042ee7a
Job Finished

Preparing to load: produkt_klima_tag_19350101_20171231_04371.txt
Starting job{} c07ebd3b-7788-4177-9353-d12f0069495f
Job Finished

Preparing to load: produkt_klima_tag_19941001_20171231_04997.txt
Starting job{} dac5b062-20c5-4f49-8b27-982e259d137f
Job Finished

Preparing to load: produkt_klima_tag_19630101_19761231_01066.txt
Starting job{} 0d6e9d8d-00ab-4514-9305-242a6535e474
Job Finished

Preparing to load

Starting job{} d0815af4-58b4-49a2-80a6-5dd9a743f017
Job Finished

Preparing to load: produkt_klima_tag_19690601_19970331_04571.txt
Starting job{} cc98adea-d91c-42e9-8f90-0d5f0f096f48
Job Finished

Preparing to load: produkt_klima_tag_19450101_20171231_01346.txt
Starting job{} ed9663c2-52e1-4234-8d86-1a4b4773e334
Job Finished

Preparing to load: produkt_klima_tag_19490101_20051231_05361.txt
Starting job{} d76343d7-0b62-4c43-bc92-ac94be8e1611
Job Finished

Preparing to load: produkt_klima_tag_19981001_20000131_06112.txt
Starting job{} d23757f9-8f2e-402c-b5cd-fcc46ac45235
Job Finished

Preparing to load: produkt_klima_tag_19470101_20171231_03679.txt
Starting job{} 75b09f01-5eb1-422d-94e1-8ec75dba0aae
Job Finished

Preparing to load: produkt_klima_tag_19470101_20171231_02201.txt
Starting job{} 05565e15-a145-4898-b4ce-6cd02f4748b5
Job Finished

Preparing to load: produkt_klima_tag_18900101_20171231_00691.txt
Starting job{} 4ef9ca79-f7c5-42af-827c-ddce4966e6a0
Job Finished

Preparing to load